In [1]:
from contextlib import contextmanager
import os
import datetime as dt
import numpy as np
import h5py
import time
import sys
os.chdir('../telescope_control/data_aquisition/')
sys.path.append('../')
sys.path.append('/software_git_repos/cofe-python-analysis-tools/utils_meinhold/')
sys.path.append('/software_git_repos/cofe-python-analysis-tools/utils_zonca/')
from glob import glob

sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../VtoT/')


import Tkinter,tkFileDialog

import cofe_util as cu
import realtime_gp as rt
import datetime

from Tkinter import *
import ttk

In [2]:
def nametochan(name):
    #function to convert channel numbers to channel names

    #names of each channel
    chans = {
        'all': 'all',  'H1HiAC':'ch0',  'H1HiDC':'ch1',
        'H1LoAC':'ch2' ,  'H1LoDC':'ch3', 'H2HiAC':'ch4' ,
        'H2HiDC':'ch5' ,  'H2LoAC':'ch6',  'H2LoDC':'ch7',
        'H3HiAC':'ch8', 'H3HiDC':'ch9',  'H3LoAC':'ch10',
         'H3LoDC':'ch11', 'HornTop':'ch12', 'Amplifier': 'ch13',
        'Cooler':'ch14', 'Transition':'ch15'}

    chan = chans[name]

    return chan

In [3]:
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [41]:
def plot_some_chans2(datatype='demod',plottype='toi',filelist=None,component='T',samprate=30,minfreq=.1):
    
    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    main = Tk()
    main.title("Choose channels to plot")
    main.geometry("+50+150")
    frame = ttk.Frame(main, padding=(3, 3, 12, 12))
    frame.grid(column=0, row=0, sticky=(N, S, E, W))

    #set up the listbox to choose what chans to plot
    chan_labels = StringVar()
    chan_labels.set("H1HiAC_T H1HiAC_Q H1HiAC_U H1HiDC H1LoAC_T H1LoAC_Q H1LoAC_U H1LoDC\
                H2HiAC_T H2HiAC_Q H2HiAC_U H2HiDC H2LoAC H2LoDC\
                 H3HiAC_T H3HiAC_Q H3HiAC_U H3HiDC H3LoAC_T H3LoAC_Q H3LoAC_U H3LoDC HornTop Amplifier Cooler Transition")           
    tlist=[]
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            h=np.float64(f[-11:-9])
            m=np.float64(f[-9:-7])
            s=np.float64(f[-7:-5])
            hf=h5py.File(f)
            t=h+m/60.+(s+(hf['demod_data']['rev']-hf['demod_data']['rev'][0])/1000.)/3600.
            tlist.append(t)  #just generate a time to plot relative to
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        h=np.float64(filelist[0][-11:-9])
        m=np.float64(filelist[0][-9:-7])
        s=np.float64(filelist[0][-7:-5])
        d=rt.demod.read_raw(filelist)
        t=h+m/60.+(s+(d['rev']-d['rev'][0])/1000.)/3600.
        tlist.append(t)
    ut=concatenate(tlist)
    #figure()
    #plot(ut)
    lstbox = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox.grid(column=0, row=0, columnspan=2)
    
    lstbox2 = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox2.grid(column=2, row=0, columnspan=2)
    #start the plot

    fname=filelist[0]
    yyyy=os.path.dirname(filelist[0])[-8:-4]
    mm=os.path.dirname(filelist[0])[-4:-2]
    dd=os.path.dirname(filelist[0])[-2:]

    def select():
        reslist = list()
        selection = lstbox.curselection()
        figure()
        if datatype=='demod':
            title('%s TOI from %s %s' %(mm,dd,yyyy))
        if datatype=='raw':
            title('TOI from %s %s %s' %(mm,dd,yyyy))

        for i in selection:
            entry = lstbox.get(i)
            reslist.append(entry)
        for val in reslist:
            chan=nametochan(val[:6])
            if datatype=='demod':
                component=val[-1]
                if ((component != 'T') and (component != 'Q') and (component != 'U')):
                    component='T'
                plot(ut,d[chan][component],label=val)
                xlabel('ut, hours')
            if datatype=='raw':
                plot(d[chan].flatten(),label=val)
                xlabel('samples')
            ylabel('Output [v]')
            legend()

    def selectfft():
        reslist2 = list()
        selection2 = lstbox2.curselection()
        figure()
        if datatype=='demod':
            title('%s ASD from %s %s %s' %(component,mm,dd,yyyy))
        if datatype=='raw':
            title('ASD from %s %s %s' %(mm,dd,yyyy))
        for i in selection2:
            entry2 = lstbox2.get(i)
            reslist2.append(entry2)
        for val in reslist2:
            chan=nametochan(val[:6])
            if datatype=='demod':
                freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
            if datatype=='raw':
                freq,psd=cu.nps(d[chan].flatten(),samprate*256,minfreq=minfreq)
            plot(freq,sqrt(psd),label=val)
            xlabel('Frequency [Hz]')
            ylabel(r'ASD [$\frac{V}{\sqrt{Hz}}$]')
            legend()
                
    btn = ttk.Button(frame, text="Select for TOI", command=select)
    btn.grid(column=1, row=1)
    
    btn2 = ttk.Button(frame, text="Select for ASD", command=selectfft)
    btn2.grid(column=3, row=1)
    
    main.mainloop()

In [5]:
def read_some_data(datadict,datalabel,datatype='demod', filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            hf=h5py.File(f)
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        d=rt.demod.read_raw(filelist)
    datadict[datalabel]=d
    return

In [6]:
c='T'
(c!='Q') or (c!='T')

True

In [7]:
def asd_from_dict(data_dict,minfreq=1.):
    #run nps on all the channels for each of the loaded data sets, return asd dictionary
    asd_dict={}
    chans=['ch%s' %str(i) for i in range(16)]
    for datset in data_dict.keys():
        asd_dict[datset]={}
        datatype='demod'
        print(datset)
        d=data_dict[datset]
        if len(d.dtype)==18:
            datatype='raw'
        samprate=1000./float(d['rev'][10]-d['rev'][9])
        for chan in chans:
            if datatype=='raw':
                freq,psd=cu.nps(d[chan].flatten(),samprate*256,minfreq=minfreq)
                asd_dict[datset]['freq']=freq
                asd_dict[datset][chan]=sqrt(psd)
            if datatype=='demod':
                asd_dict[datset][chan]={}
                for component in ['T','Q','U']:
                    freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
                    asd_dict[datset][chan][component]=sqrt(psd)
                    asd_dict[datset]['freq']=freq
                    
    return asd_dict            

In [8]:
def plot_asd_dict(asd_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=None,kperv=[1]):
    #function to give a choice of preloaded datasets from dictionary data_dict
    figure()
    if datalist==None:
        datalist=asd_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in datalist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    
    count = 0
    for dname in datalist:
        for channame,component in zip(chanlist,componentlist):
            try:
                asd=asd_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
            except:
                asd=asd_dict[dname][nametochan(channame)]           #if it's raw data so not a dict
            plot(asd_dict[dname]['freq'],asd*kperv[count],label=dname+ '  '+channame+' '+component,lw=2)
            if (count+1) < len(kperv):
                count += 1
    legend()
    xlabel('Frequency [Hz]')
    if len(kperv) == 1 and kperv[0] == 1:
        ylabel(r'ASD [$\frac{V}{\sqrt{Hz}}$]')
    else:
        ylabel(r'ASD [$\frac{T}{\sqrt{Hz}}$]')

In [9]:
def plot_toi_dict(toi_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=None):
    #function to give a choice of preloaded datasets from dictionary data_dict
    figure()
    if datalist==None:
        datalist=toi_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in chanlist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    titletxt=[]
    for dname in datalist:
        samprate=1000./(toi_dict[dname]['rev'][1]-toi_dict[dname]['rev'][0])
        print(dname,samprate)
        titletxt.append(dname)
        for channame,component in zip(chanlist,componentlist):
            print(channame,component)
            try:
                toi=toi_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/samprate
                plot(t,toi,label=dname+ '  '+channame+' '+component,lw=2)
            except:
                toi=toi_dict[dname][nametochan(channame)].flatten()#if it's raw data so not a dict
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/(256.*samprate)
                plot(t,toi,label=dname+ '  '+channame+' '+component,lw=2)
            
    title(titletxt)
    legend()
    xlabel('Time [S]')
    ylabel(r'Signl [V]')

In [10]:
def plot_nl_dict(toi_dict,chanlist=['H1HiDC'],componentlist=['T'],datalist=None):
    #function to give a choice of preloaded datasets from dictionary data_dict
    #this plots chosen channels vs H1LoDC (should be attenuated more linear detector)
    figure()
    if datalist==None:
        datalist=toi_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in chanlist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    titletxt=[]
    for dname in datalist:
        samprate=1000./(toi_dict[dname]['rev'][1]-toi_dict[dname]['rev'][0])
        print(dname,samprate)
        titletxt.append(dname)
        for channame,component in zip(chanlist,componentlist):
            print(channame,component)
            try:
                toi=toi_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
                toi_x=toi_dict[dname][nametochan('H2LoDC')][component] #highly attenuated channel
                plot(toi_x-max(toi_x),toi-max(toi),label=dname+ '  '+channame+' '+component)
            except:
                toi=toi_dict[dname][nametochan(channame)].flatten()#if it's raw data so not a dict
                toi_x=toi_dict[dname][nametochan('H2LoDC')].flatten()# highly attenuated channel
                plot(toi_x-max(toi_x),toi-max(toi),label=dname+ '  '+channame+' '+component)
            
    title(titletxt)
    legend()
    xlabel('H1LoDC [V]')
    ylabel(r'Signal [V]')

In [11]:
ddict={}

In [25]:
read_some_data(ddict,'sky',datatype='raw')

1000
16


In [64]:
plot_some_chans2(datatype='raw', component='T')

1000
16


In [54]:
chan = 'H3HiDC'
skylev = ddict['sky'][nametochan(chan)].mean()
amblev = ddict['amb'][nametochan(chan)].mean()

print skylev
print amblev

gain = (300 - 20)/(amblev - skylev)
print 'gain: ', gain

y = amblev/skylev
Th = 300.
Tc = 20.
Tsys = (Th - y*Tc)/(y-1)
print y
print Tsys

-0.0780620615564
-0.969179183066
gain:  -314.21234453
12.4154956165
4.52806338045


In [41]:
from collections import defaultdict

H = defaultdict(dict)


for h in range(1,4):

    channame='H%dHiDC' % h
    Horn = (ddict['H%d_cal_3_29' % h][nametochan(channame1)])
    H[h] = []
    for r in range(len(Horn)):
        H[h].append(Horn[r].mean())
        
    H[h] = np.asarray(H[h])
        
rev = np.arange(len(H[1]))


In [38]:
h = 1
plot(rev, H[h], lw=2)

In [20]:
asd_dict=asd_from_dict(ddict, minfreq=10)

amb_3_23
sky_3_23


In [26]:
plot_asd_dict(asd_dict,chanlist=['H1HiAC'], kperv = [1.29,1.31,1.16])

C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [10]:
rdict={}

In [11]:
read_some_data(rdict,'nl_8_3_20',datatype='raw')

1001
16


In [31]:
figure()
plot(rdict['nl_8_3_20']['ch0'].flatten(),rdict['nl_8_3_20']['ch1'].flatten())

In [102]:
plot_toi_dict(rdict,chanlist=['H1HiDC','H1LoDC','H2LoDC'],componentlist=['T'],datalist=['nl_8_3_20'])

('nl_8_3_20', 31.25)
('H1HiDC', 'T')
('H1LoDC', 'T')
('H2LoDC', 'T')


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [103]:
plot_nl_dict(rdict,chanlist=['H1HiDC', 'H1LoDC'],componentlist=['T'],datalist=['nl_8_3_20'])

('nl_8_3_20', 31.25)
('H1HiDC', 'T')


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:22: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


('H1LoDC', 'T')


In [23]:
ddict.keys()

['H3_cal_3_29']

In [172]:
h1dclevels_3_16={}
channame1='H1HiDC'
channame2='HornTop'
var = 'T'
H1=(ddict['amb_3_16'][nametochan(channame1)].flatten())[var]
Input = (ddict['amb_3_16'][nametochan(channame2)].flatten())[var]
x = np.arange(len(H1))

In [173]:
def smoothTriangle(data, degree, dropVals=False):
    triangle=np.array(range(degree) + [degree] + range(degree)[::-1]) + 1
    smoothed=[]

    for i in range(degree, len(data) - degree * 2):
        point=data[i:i + len(triangle)] * triangle
        smoothed.append(sum(point)/sum(triangle))
    if dropVals:
        return smoothed
    smoothed=[smoothed[0]]*(degree + degree/2) + smoothed
    while len(smoothed) < len(data):
        smoothed.append(smoothed[-1])
    return smoothed

In [174]:
H1smoothed=smoothTriangle(H1, 20)  # setting degree to 10

In [175]:
der = np.diff(H1smoothed) / np.diff(x)
x2 = (x[:-1] + x[1:]) / 2

der2 = np.diff(der) / np.diff(x2)
x3 = (x2[:-1] + x2[1:]) / 2

In [150]:
#plot smoothed curve + derivatives
#plt.plot(x, H1, label = 'data')
plt.plot(x, H1smoothed, label = 'smoothed data')
plt.plot(x2, der*20, label = '1st derivative x 20')
#plt.plot(x3, der2*100, label='2nd derivative x 100')
plt.ylabel('V')
plt.legend()

In [162]:
idx = np.where(abs(der) < 5e-5)[0]

In [183]:
idxP = []
idxN = []
idx = 0
for i in range(len(der)):
    if der[i] > 5e-5:
        idxP.append(np.arange(idx,i))
        idx = i
    if der[i] < -5e-5:
        idxN.append(np.arange(idx,i))
        idx = i

In [184]:
idxP.flatten()

AttributeError: 'list' object has no attribute 'flatten'

In [177]:

plt.plot(x, H1, label = 'data')
plt.plot(x[idx], H1[idx], label = 'level region')
plt.plot(x, H1smoothed, label = 'smoothed data')
plt.plot(x2, der*20, label = '1st derivative x 20')
plt.legend()

C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:3: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
  app.launch_new_instance()


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [164]:
InputV = []
H1V = []
for i in range(len(idx)):
    InputV.append(Input[idx[i]].mean())
    H1V.append(H1[idx[i]].mean())
    

H1V = [x for _,x in sorted(zip(InputV,H1V))]
InputV = sorted(InputV)

In [166]:
plt.plot(InputV, H1V, lw=2)
plt.ylabel('input voltage')
plt.xlabel('output voltage')

In [45]:
h1dclevels_3_14={}
channame='H1HiDC'
for key in ddict.keys():
    toi=ddict[key][nametochan(channame)].flatten()
    h1dclevels_3_14[key]=np.mean(toi)
    

In [59]:
print h1dclevels_3_14
y=h1dclevels_3_14['amb_3_14']/h1dclevels_3_14['LN_3_14']
print('y= ',y)
g=(290.-77.)/(h1dclevels_3_14['amb_3_14']-h1dclevels_3_14['LN_3_14'])
print('g = ',g,'K/V')
h1tsys_3_14=(290.-77.*y)/(y-1)
print(h1tsys_3_14)

{'LN_3_14': -0.23080378270149232, 'amb_3_14': -0.59305374652147291, 'sky_3_15': -0.071202472050984705, 'amb_3_15': -0.58871274590492251}
('y= ', 2.5695148475469001)
('g = ', -587.99177715266785, 'K/V')
58.7107263642


In [109]:
h1dclevels_3_15={}
channame='H1HiDC'
for key in ddict.keys():
    toi=ddict[key][nametochan(channame)].flatten()
    h1dclevels_3_15[key]=np.mean(toi)
    

In [110]:
print h1dclevels_3_15
y=h1dclevels_3_15['amb_3_15']/h1dclevels_3_15['sky_3_15']
print('y= ',y)
g= -588
Tsky = 293.35 - (h1dclevels_3_15['amb_3_15'] - h1dclevels_3_15['sky_3_15'])*g
print('Tsky = ',Tsky,'K')
h1tsys_3_15=(293.35-Tsky*y)/(y-1)
print(h1tsys_3_15)

{'amb_3_15_Vg1': -0.2413669200738271, 'sky_3_15_Vg1': -0.025998044808705649, 'sky_3_15': -0.071202472050984705, 'amb_3_15': -0.58871274590492251}
('y= ', 8.2681503738152973)
('Tsky = ', -10.946041026115381, 'K')
52.8130945921


In [298]:
dname = 'LN_3_13'
ACchanname = 'H3HiAC'
DCchanname = 'H3HiDC'
component = 'T'

asd = asd_dict[dname][nametochan(ACchanname)]
freq = asd_dict[dname]['freq']
toi = ddict[dname][nametochan(DCchanname)].flatten()
samprate=1000./(ddict[dname]['rev'][1]-ddict[dname]['rev'][0])
t = ddict[dname]['azi'][0]+arange(len(toi))/(256.*samprate)

In [299]:
iwng = set(np.where(freq >= 150.)[0])
iwnl = set(np.where(freq <= 900.)[0])

iwn = list(iwng.intersection(iwnl))

wn = asd[iwn]

In [300]:
dcl = toi.mean()
wnl = wn.mean()
#dcrms = np.sqrt(np.mean(toi**2))
dcrms = 2*(abs(dcl - toi)).mean()
beta = (dcl*100/wnl)**2 /1e9

print 'DC level: ', dcl
print 'WN level: ', wnl
print 'DC RMS: ', dcrms
print 'Bandwidth(GHz): ', beta

DC level:  -0.14921598355
WN level:  0.00162633860461
DC RMS:  0.00865078915838
Bandwidth(GHz):  0.0841799667244


In [293]:
plt.plot(freq, asd, 'b-')
plt.plot(freq[iwn], wn, 'g-')

In [301]:
plt.plot(t, toi)

In [276]:
vh = .0034
vc = .0016
th = 290
tc = 77

y = vh/vc

Tsys = (th - y*tc)/(y-1)
print y
print Tsys

2.125
112.333333333
